## Groundwater Whirls with PRT

This is a 10 layer steady-state problem involving anisotropic groundwater
flow.  The XT3D formulation is used to represent variable hydraulic
conductivitity ellipsoid orientations.  The resulting flow pattern consists
of groundwater whirls, as described in the XT3D documentation report. A
particle tracking model is created.

### Initial setup

Import dependencies, define the example name and workspace, and read settings from environment variables.

In [ ]:
import pathlib as pl
from itertools import repeat

import flopy
import pandas as pd
import git
import numpy as np
from flopy.mf6 import MFSimulation
from modflow_devtools.misc import get_env

# Example name and workspace paths. If this example is running
# in the git repository, use the folder structure described in
# the README. Otherwise just use the current working directory.
sim_name = "whirls-prt"
gwf_name = sim_name + "-gwf"
prt_name = sim_name + "-prt"
try:
    root = pl.Path(git.Repo(".", search_parent_directories=True).working_dir)
except:
    root = None
workspace = root / "examples" if root else pl.Path.cwd()
figs_path = root / "figures" if root else pl.Path.cwd()

sim_ws = workspace / sim_name

# Define output file names
headfile_gwf = f"{gwf_name}.hds"
budgetfile_gwf = f"{gwf_name}.cbb"
budgetfile_prt = f"{prt_name}.cbb"
trackfile_prt = f"{prt_name}.trk"
trackhdrfile_prt = f"{prt_name}.trk.hdr"
trackcsvfile_prt = f"{prt_name}.trk.csv"

# Settings from environment variables
write = get_env("WRITE", True)
run = get_env("RUN", True)
plot = get_env("PLOT", True)
plot_show = get_env("PLOT_SHOW", True)
plot_save = get_env("PLOT_SAVE", True)

### Define parameters

Define model units, parameters and other settings.

In [ ]:
# Model units
length_units = "meters"
time_units = "days"

# Model parameters
nper = 1  # Number of periods
nlay = 10  # Number of layers
nrow = 10  # Number of rows
ncol = 51  # Number of columns
delr = 100.0  # Spacing along rows ($m$)
delc = 100.0  # Spacing along columns ($m$)
top = 0.0  # Top of the model ($m$)
botm_str = "-100, -200, -300, -400, -500, -600, -700, -800, -900, -1000"  # Layer bottom elevations ($m$)
strt = 0.0  # Starting head ($m$)
icelltype = 0  # Cell conversion type
k11 = 1.0  # Hydraulic conductivity in the 11 direction ($m/d$)
k22 = 0.1  # Hydraulic conductivity in the 22 direction ($m/d$)
k33 = 1.0  # Hydraulic conductivity in the 33 direction ($m/d$)
angle1_str = "45, 45, 45, 45, 45, -45, -45, -45, -45, -45"  # Rotation of the hydraulic conductivity ellipsoid in the x-y plane
inflow_rate = 0.01  # Inflow rate ($m^3/d$)

# Static temporal data used by TDIS file
# Simulation has 1 steady stress period (1 day)
perlen = [1.0]
nstp = [1]
tsmult = [1.0]
tdis_ds = list(zip(perlen, nstp, tsmult))

# Parse strings into lists
botm = [float(value) for value in botm_str.split(",")]
angle1 = [float(value) for value in angle1_str.split(",")]

# Solver settings
nouter = 50
ninner = 100
hclose = 1e-9
rclose = 1e-6

### Model setup

Build models, write input files, and run the simulation.

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=sim_name, sim_ws=sim_ws, exe_name="mf6")
flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_ds, time_units=time_units)
ims = flopy.mf6.ModflowIms(
    sim,
    linear_acceleration="bicgstab",
    outer_maximum=nouter,
    outer_dvclose=hclose,
    inner_maximum=ninner,
    inner_dvclose=hclose,
    rcloserecord=f"{rclose} strict",
)
gwf = flopy.mf6.ModflowGwf(sim, modelname=gwf_name, save_flows=True)
sim.register_solution_package(ims, [gwf.name])
flopy.mf6.ModflowGwfdis(
    gwf,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
flopy.mf6.ModflowGwfnpf(
    gwf,
    icelltype=icelltype,
    k=k11,
    k22=k22,
    k33=k33,
    angle1=angle1,
    save_specific_discharge=True,
    save_saturation=True,
    save_flows=True,
    xt3doptions=True,
)
flopy.mf6.ModflowGwfic(gwf, strt=strt)
rate = np.zeros((nlay, nrow, ncol), dtype=float)
rate[:, :, 0] = inflow_rate
rate[:, :, -1] = -inflow_rate
wellay, welrow, welcol = np.where(rate != 0.0)
wel_spd = [((k, i, j), rate[k, i, j], 1 if rate[k, i, j] > 0 else 3) for k, i, j in zip(wellay, welrow, welcol)]
wel_spd = {0: wel_spd}
flopy.mf6.ModflowGwfwel(
    gwf,
    stress_period_data=wel_spd,
    pname="WEL",
    auxiliary=["IFLOWFACE"]
)
flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=[headfile_gwf],
    budget_filerecord=[budgetfile_gwf],
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

# Instantiate the MODFLOW 6 prt model
prt = flopy.mf6.ModflowPrt(
    sim, modelname=prt_name, model_nam_file="{}.nam".format(prt_name)
)

# Instantiate the MODFLOW 6 prt discretization package
flopy.mf6.ModflowGwfdis(
    prt,
    length_units=length_units,
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)

def get_izone():
    izone = []

    # zone 1 is the default (non-terminating)
    def layer():
        a = np.ones((prt.modelgrid.nrow, prt.modelgrid.ncol), dtype=np.int32)
        a[:,-1] = 2
        return a

    for _ in range(10):
        izone.append(layer())

    return izone

# Instantiate the MODFLOW 6 prt model input package
izone = get_izone()
flopy.mf6.ModflowPrtmip(prt, pname="mip", porosity=0.1) #, izone=izone)

def intersect(p):
    try:
        return prt.modelgrid.intersect(*p)
    except:
        import pdb; pdb.set_trace()

xs = np.array(list(repeat(50, 4)))
ys = np.linspace(1, 999, 4)
zs = np.linspace(1, 999, 4) * -1
points = np.transpose(np.array(np.meshgrid(xs, ys, zs)).reshape(3, -1))
releasepts = [
    (i, *intersect(p), *p) for i, p in enumerate(points)
]

flopy.mf6.ModflowPrtprp(
    prt,
    nreleasepts=len(releasepts),
    packagedata=releasepts,
    perioddata={0: ["FIRST"]},
    exit_solve_tolerance=1e-5,
    istopzone=2
)

flopy.mf6.ModflowPrtoc(
    prt,
    pname="oc",
    budget_filerecord=[budgetfile_prt],
    track_filerecord=[trackfile_prt],
    trackcsv_filerecord=[trackcsvfile_prt],
    saverecord=[("BUDGET", "ALL")],
)

flopy.mf6.ModflowGwfprt(sim, exgtype="GWF6-PRT6", exgmnamea=gwf_name, exgmnameb=prt_name)

# Create an explicit model solution (EMS) for the MODFLOW 6 prt model
ems = flopy.mf6.ModflowEms(
    sim,
    pname="ems",
    filename="{}.ems".format(prt_name),
)
sim.register_solution_package(ems, [prt.name])

In [ ]:

# Write and run the simulation.
sim.write_simulation(silent=False)
success, buff = sim.run_simulation(silent=False)


In [ ]:

pls = pd.read_csv(sim_ws / trackcsvfile_prt)

import pyvista as pv
from flopy.export.vtk import Vtk

gwf = sim.get_model(gwf_name)
pv.set_plot_theme("document")
axes = pv.Axes(show_actor=False, actor_scale=2.0, line_width=5)
vert_exag = 1
vtk = Vtk(model=gwf, binary=False, vertical_exageration=vert_exag, smooth=False)
vtk.add_model(gwf)
vtk.add_pathline_points(pls)
gwf_mesh, prt_mesh = vtk.to_pyvista()
wel_nodes = gwf.modelgrid.get_node([w[0] for w in wel_spd[0]])
wel_mesh = gwf_mesh.remove_cells(
    list(set(range(gwf.modelgrid.nnodes)) - set(wel_nodes)), inplace=False
)
# eps_mesh = prt_mesh.remove_cells(
#     list(set(range(gwf.modelgrid.nnodes)) - set(wel_nodes)), inplace=False
# )

p = pv.Plotter(
    window_size=[500, 500],
    notebook=False,
)
p.enable_anti_aliasing()
p.add_mesh(
    gwf_mesh,
    opacity=0.1,
    style="wireframe",
)
p.add_mesh(
    wel_mesh,
    opacity=0.1,
    color="red",
)
p.add_mesh(
    prt_mesh,
    opacity=0.3,
    point_size=4,
    line_width=3,
    render_points_as_spheres=True,
    render_lines_as_tubes=True,
    smooth_shading=True,
)
p.add_mesh(
    pls[pls.ireason == 0][["x", "y", "z"]].to_numpy(),
    color="green",
    point_size=8,
    render_points_as_spheres=True
)
p.add_mesh(
    pls[pls.ireason == 3][["x", "y", "z"]].to_numpy(),
    color="red",
    point_size=8,
    render_points_as_spheres=True
)
p.add_legend(
    labels=[("Wells", "red")],
    bcolor="white",
    face="r",
    size=(0.1, 0.1),
)
p.camera.zoom(2)
p.show()